In [57]:
from scipy.io import loadmat
from os.path import abspath
import numpy as np
from numpy._distributor_init import NUMPY_MKL

In [100]:
def load_data(path):
    input_file = open(path)
    output_data = []
    for line in input_file:
        one_row = line.split(',')
        output_data.append([float(x) for x in one_row])

    print('Input data info.')
    print('\tFeatures amount: %d, set size %d' % (len(output_data[0]), len(output_data)))
    return np.array(output_data)

In [124]:
def show_prediction(output_vector):
    index_of_max = np.argmax(np.roll(output_vector[1:],1))
    return index_of_max

In [59]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [67]:
# sum of all pairs theta[i]*input[i] in one neuron
# z(i) = sum(theta * x) = theta' * x
def unit_summarization(theta, input_vec):
    return (np.array(theta) * np.array(input_vec)).sum()

In [68]:
# computing the activation function of neuron
# a(i) = g(z(i))
def unit_activation(theta, input_vec):
    sum_result = unit_summarization(theta, input_vec)
    return sigmoid(sum_result)

In [83]:
# computing the activation vector of whole network layer
# A = g(z(i)) for all units on layer
# [1] - additional neuron (bias)
def forward_propagation_step(theta_matrix, input_vec):
    activations_vector = []
    for i in range(len(theta_matrix)):
        activations_vector.append(unit_activation(theta_matrix[i], input_vec))
    return np.concatenate(([1], activations_vector))

In [113]:
def forward_propagation(theta_one, theta_two, input_vec):
    # print('Computing input level activation vector...')
    activation_vector_first_layer = np.concatenate(([1], input_vec))
    # print('Computing second level activation vector...')
    activation_vector_second_layer = forward_propagation_step(theta_one, activation_vector_first_layer)
    # print('Computing output level activation vector...')
    activation_output = forward_propagation_step(theta_two, activation_vector_second_layer)
    return activation_output

In [114]:
data_x = load_data(abspath('demo/init/coursera/neuralnetwork/data/data_x.csv'))
data_y = load_data(abspath('demo/init/coursera/neuralnetwork/data/data_y.csv'))
theta_one = load_data(abspath('demo/init/coursera/neuralnetwork/data/theta1.csv'))
theta_two = load_data(abspath('demo/init/coursera/neuralnetwork/data/theta2.csv'))

Input data info.
	Features amount: 400, set size 5000
Input data info.
	Features amount: 1, set size 5000
Input data info.
	Features amount: 401, set size 25
Input data info.
	Features amount: 26, set size 10


In [128]:
for i in range(10):
    print('Real digit:      %d\n'
          'Predicted digit: %d\n%s' 
          % (i, show_prediction(forward_propagation(theta_one, theta_two, data_x[i * 500])), '='*10))

Real digit:      0
Predicted digit: 0
Real digit:      1
Predicted digit: 1
Real digit:      2
Predicted digit: 2
Real digit:      3
Predicted digit: 3
Real digit:      4
Predicted digit: 4
Real digit:      5
Predicted digit: 5
Real digit:      6
Predicted digit: 6
Real digit:      7
Predicted digit: 7
Real digit:      8
Predicted digit: 8
Real digit:      9
Predicted digit: 9
